In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip drive/My\ Drive/train.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/cat.183.jpg       
  inflating: __MACOSX/train/._cat.183.jpg  
  inflating: train/cat.2677.jpg      
  inflating: __MACOSX/train/._cat.2677.jpg  
  inflating: train/cat.6411.jpg      
  inflating: __MACOSX/train/._cat.6411.jpg  
  inflating: train/dog.4744.jpg      
  inflating: __MACOSX/train/._dog.4744.jpg  
  inflating: train/cat.9722.jpg      
  inflating: __MACOSX/train/._cat.9722.jpg  
  inflating: train/dog.10373.jpg     
  inflating: __MACOSX/train/._dog.10373.jpg  
  inflating: train/dog.2335.jpg      
  inflating: __MACOSX/train/._dog.2335.jpg  
  inflating: train/dog.2453.jpg      
  inflating: __MACOSX/train/._dog.2453.jpg  
  inflating: train/cat.7069.jpg      
  inflating: __MACOSX/train/._cat.7069.jpg  
  inflating: train/dog.10415.jpg     
  inflating: __MACOSX/train/._dog.10415.jpg  
  inflating: train/dog.3995.jpg      
  inflating: __MACOSX/train/._dog.3995.jpg  
  inflating: train/dog.4022.jpg    

In [ ]:
!unzip drive/My\ Drive/test1.zip

Streaming output truncated to the last 5000 lines.
  inflating: test1/7272.jpg          
  inflating: __MACOSX/test1/._7272.jpg  
  inflating: test1/8141.jpg          
  inflating: __MACOSX/test1/._8141.jpg  
  inflating: test1/1603.jpg          
  inflating: __MACOSX/test1/._1603.jpg  
  inflating: test1/397.jpg           
  inflating: __MACOSX/test1/._397.jpg  
  inflating: test1/5465.jpg          
  inflating: __MACOSX/test1/._5465.jpg  
  inflating: test1/3014.jpg          
  inflating: __MACOSX/test1/._3014.jpg  
  inflating: test1/11553.jpg         
  inflating: __MACOSX/test1/._11553.jpg  
  inflating: test1/10895.jpg         
  inflating: __MACOSX/test1/._10895.jpg  
  inflating: test1/5471.jpg          
  inflating: __MACOSX/test1/._5471.jpg  
  inflating: test1/10659.jpg         
  inflating: __MACOSX/test1/._10659.jpg  
  inflating: test1/3000.jpg          
  inflating: __MACOSX/test1/._3000.jpg  
  inflating: test1/11547.jpg         
  inflating: __MACOSX/test1/._11547.jpg 

In [ ]:
import cv2
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard

import glob
from tqdm import tqdm
import warnings;

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

warnings.filterwarnings('ignore')

np.set_printoptions(threshold=sys.maxsize)

# Downloading dataset

#train_x = glob.glob('train/*.jpg')
#val_x = glob.glob('test1/*.jpg')

# Loading and preprocessing images
#all_images = []
#all_images2 = []


#train_size = np.array(train_x)


#test_size = np.array(val_x)

# Splitting data into training and validation data
#train_x, val_x = train_test_split(all_images, random_state=32, test_size=0.1)
from PIL import Image
training_data = []
labels = []
files = glob.glob('train/*.jpg')
for filepath in files:
    image = Image.open(filepath)
    image = image.resize((28, 28))
    training_data.append(np.array(image))
    if "cat" in filepath:
        labels.append(0)
    elif "dog" in filepath:
        labels.append(1)

training_data = np.array(training_data)
training_data = np.resize(training_data, (len(labels), 28, 28, 3))
training_data = training_data.astype('float32')

labels = np.array(labels)
labels = tensorflow.keras.utils.to_categorical(labels, 2)

In [ ]:


# Defining function pixalate_image to reduce image resolution while keeping the image size constant
def pixalate_image(image, scale_percent=40):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)

    small_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

    width = int(small_image.shape[1] * 100 / scale_percent)
    height = int(small_image.shape[0] * 100 / scale_percent)
    dim = (width, height)

    low_res_image = cv2.resize(small_image, dim, interpolation=cv2.INTER_AREA)

    return low_res_image

# Low resolution images for the training set
train_x_px = []

for i in range(train_x.shape[0]):
    temp = pixalate_image(train_x[i, :, :, :])
    train_x_px.append(temp)

train_x_px = np.array(train_x_px)

# Low resolution images for the validation set
val_x_px = []

for i in range(val_x.shape[0]):
    temp = pixalate_image(val_x[i, :, :, :])
    val_x_px.append(temp)

val_x_px = np.array(val_x_px)

Input_img = Input(shape=(80, 80, 3))



NameError: ignored

In [ ]:

# Encoding architecture
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x2 = MaxPool2D((2, 2))(x2)
encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

# Decoding architecture
x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x3 = UpSampling2D((2, 2))(x3)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
decoded = Conv2D(3, (3, 3), padding='same')(x1)

autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=2, mode='auto')

a_e = autoencoder.fit(train_x_px, train_x,
                      epochs=5,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(val_x_px, val_x),
                      callbacks=[early_stopper])

predictions = autoencoder.predict(val_x_px)


NameError: ignored

In [ ]:
n = 3
plt.figure(figsize=(20, 10))

for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(val_x_px[i + 20])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(predictions[i + 20])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()



In [ ]:


n = 3
fig = plt.figure(figsize=(20, 10))

for i in range(n):
    ax = fig.add_subplot(2, n, i + 1)
    ax.imshow(val_x_px[i + 20])
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_xticks([])
    ax.set_yticks([])

    ax = fig.add_subplot(2, n, i + 1 + n)
    ax.imshow(predictions[i + 20])
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_xticks([])
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()
